In [1]:
import pandas as pd
import sys
import os
parent_dir = os.path.dirname(os.path.abspath(''))
sys.path.append(parent_dir)

from utils import const

data_path = "../data/binance/BTCUSDT-1m-2024-all-with-indicators.csv"

df = pd.read_csv(data_path)
df["open_time"] = pd.to_datetime(df["open_time"], unit="ms")
df = df.set_index("open_time")

In [ ]:
df["target"] = (df["close"].shift(-1) > df["close"]).astype(int)
df = df.dropna()

X = df[const.factor_cols]
y = df["target"]

X_train = X[X.index < const.split_time]
X_test = X[X.index >= const.split_time]
y_train = y[y.index < const.split_time]
y_test = y[y.index >= const.split_time]

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

estimator = RandomForestClassifier(random_state=42)
selector = RFE(estimator, n_features_to_select=1, step=1)
selector = selector.fit(X_train, y_train)

ranking = selector.ranking_
feature_ranking = sorted(zip(const.factor_cols, ranking), key=lambda x: x[1])

for feature, rank in feature_ranking:
    print(f"{feature}: rank {rank}")